# Objective
Simulate the rotor allocation for quadrotor in the event of single rotor failure (in eq.16) in arducopter SITL using Dronekit API

In [ ]:
# Mavproxy command for running udp <--> udp connection
# mavproxy.exe --master tcp:127.0.0.1:5760 --sitl 127.0.0.1:5501 --out 127.0.0.1:14550 --out 127.0.0.1:14551 --baudrate 921600

In [ ]:
# Mavproxy command for running com port <--> udp connection
# Windows
# mavproxy.exe --master com3 --sitl 127.0.0.1:5501 --out 127.0.0.1:14550 --out 127.0.0.1:14551 --baudrate 921600
# Linux
# mavproxy.exe --master \dev\ttyUSB0,57600 --sitl 127.0.0.1:5501 --out 127.0.0.1:14550 --out 127.0.0.1:14551 --baudrate 921600

### What the problem is
-

### How the problem is solved/ 1-motor failure is simulated
-

### Tests done/ what doesn't work
-

### Dummy code/function to test eq(16) motor PWM values to Uf, Tq, Tr (instantaneous):

In [5]:
from scipy.interpolate import interp1d
import numpy as np

pwm_1 = 1800
pwm_3 = 1800
pwm_4 = 1800

## Mappings for thrust to force
## Used a sample thruster/motor for force calculation:
## https://uav-en.tmotor.com/html/2018/u_0222/3.html
## Prop size: 12*4CF
# 50% - 350 G
# 65% - 550 G
# 75% - 700 G
# 85% - 870 G
# 100% - 1000 G
x =  [1000, 1500, 1650, 1750, 1850, 2000]
y = [0, 350, 550, 700, 870, 1000]

# Function that maps motor PWM to Force in grams
pwm_to_force = interp1d(x, y, 'cubic')

## Since motor uses 12inch prop, min. required arm length = root(2) * half (arm length) = 1.414 * 100 cm - 30 > 0
## arm length = 100 cm gives ~10cm + space
# Arm Length (l)
l = arm_length = 1
# Drag to Thrust co-efficients of the blade (12*4CF)
d = drag_thrust_co_efficient = 4

left_matrix = np.array([[1, 1, 1], [-l, l, 0], [d, d, -d]])
right_matrix = np.array([pwm_to_force(pwm_1), pwm_to_force(pwm_3), pwm_to_force(pwm_4)])
force_matrix = np.dot(left_matrix,right_matrix)

print(force_matrix)

uf = force_matrix[0]
tq = force_matrix[1]
tr = force_matrix[2]

print("uf: ", uf)
print("tq: ", tq)
print("tr: ", tr)

[2356.94144661    0.         3142.58859548]
uf:  2356.941446613089
tq:  0.0
tr:  3142.5885954841183
